In [38]:
%load_ext autoreload
%autoreload 2

%load_ext rpy2.ipython

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [39]:
import pandas as pd

from src.report import conf_int, t_test, f_test, general, general_assessment, other
from src.stats import LinearRegression, add_constant

# Считывание данных

In [40]:
columns = {
    'Сводные данные о продаже наличной иностранной '
    'валюты кредитными организациями физическим лицам (x1)': 'x1',
    'Курс рубля к доллару США (x2)': 'x2',
    'Цена нефти марки "Юралс" (x3)': 'x3',
    'Динамика потребительских цен по группам товаров и услуг '
    '(месяц к соответствующему месяцу предыдущего года,%)  (x4)': 'x4',
    'Объем денежных накоплений в РФ миллиард руб. (y)': 'y'
}

In [41]:
df = pd.read_excel('data/task.xlsx', sheet_name='Лист1', header=3, usecols=[1, 2, 3, 4, 5])
df = df.rename(columns=columns)

In [42]:
y = df['y']
multiple_params = ['x1', 'x2', 'x3', 'x4']
x = add_constant(df[multiple_params])

# Множественная линейная регрессия

## R

In [43]:
%Rpush df

In [44]:
%%R

library(lmtest)

m = lm(y ~ 1 + x1 + x2 + x3 + x4, data = df)

sm = summary(m)

sm


Call:
lm(formula = y ~ 1 + x1 + x2 + x3 + x4, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-953.50 -333.93   -9.69  187.71 1292.08 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)   
(Intercept) 10399.9333  6169.6976   1.686  0.11013   
x1              0.4838     0.1964   2.463  0.02473 * 
x2            130.3151   132.1259   0.986  0.33782   
x3             -6.4678    25.6981  -0.252  0.80430   
x4           -276.3727    78.9464  -3.501  0.00274 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 586.3 on 17 degrees of freedom
Multiple R-squared:  0.8064,	Adjusted R-squared:  0.7608 
F-statistic:  17.7 on 4 and 17 DF,  p-value: 6.833e-06



## Python

In [45]:
model = LinearRegression(y, x)

In [46]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.806
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     17.70
Date:                Sat, 09 Oct 2021   Prob (F-statistic):           6.83e-06
Time:                        04:37:31   Log-Likelihood:                -168.61
No. Observations:                  22   AIC:                             347.2
Df Residuals:                      17   BIC:                             352.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.04e+04   6169.698      1.686      0.1

## Модель

In [47]:
general(model)

$ y_i = a + b_{1} x_{i1} + b_{2} x_{i2} + b_{3} x_{i3} + b_{4} x_{i4} + e_i $<br>
<br>
$ \hat{y_i} = \hat{a} + \hat{b_{1}} x_{i1} + \hat{b_{2}} x_{i2} + \hat{b_{3}} x_{i3} + \hat{b_{4}} x_{i4} $<br>
<br>
$ \hat{y_i} = 10399.933 + 0.484 x_{i1} + 130.315 x_{i2} + -6.468 x_{i3} + -276.373 x_{i4} $<br>
<br>
$ S_\hat{a} = 6169.698 $<br>
$ S_{\hat{b_{1}}} = 0.196 $<br>
$ S_{\hat{b_{2}}} = 132.126 $<br>
$ S_{\hat{b_{3}}} = 25.698 $<br>
$ S_{\hat{b_{4}}} = 78.946 $

## Общая оценка

In [48]:
general_assessment(model)

### Оценка среднеквадратического отклонения возмущений.<br>
<br>
$ \sqrt{\frac{1}{n - k - 1} \sum_{i=1}^{n}{(y_i - \hat{y_i}})^2} $<br>
<br>
$ S_{e} = 586.321 $<br>
$ 2 \% < 4.20_{S_{ei}} \% < 5 \% \rightarrow $ оценка: <b>8/10</b>.<br>
<br>
<br>
### Коэффициент детерминации.<br>
<br>
$ R^2 = 1 - \frac{\sum_{i=1}^{n}{(y_i - \hat{y_i})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = \frac{\sum_{i=1}^{n}{(\hat{y_i} - \bar{y})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = 1 - \frac{RSS}{TSS} = \frac{ESS}{TSS} $<br>
<br>
$ R^2 = 0.806 $<br>
$ 0.8 < 0.806 < 0.9 \rightarrow $ оценка: <b>8/10</b>.<br>
<br>
<br>
### Средняя относительная ошибка аппроксимации.<br>
<br>
$ A = \frac{1}{n} \sum_{i=1}^{n}{|\frac{y_i - \hat{y_i}}{y_i}|} \cdot 100 \% $<br>
<br>
$ A = 2.597 $<br>
$ 2 \% < 2.597 < 5 \% \rightarrow $ оценка: <b>8/10</b>.

## Оценка модели на значимость

In [49]:
f_test(model)

### F-критерий Фишера.
<br><br>
$ H_0: b_1 = ... = b_k = 0, \\ H_1: b_1^2 + ... + b_k^2 > 0. $
<br><br>
$$ F = \frac{\frac{R^2}{k}}{\frac{1 - R^2}{(n - k - 1)}} $$
<br><br>
$ F_{набл} = 17.698 $<br>
$ p\text{-}value_F = 0.000 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 2.208 $
<br><br>
$ 6.83e-06 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - модель в целом <b>значима</b>.

## Оценка значимости параметров модели

In [50]:
t_test(model)

### t-критерий Стьюдета для оценки значимости параметров модели.
<br><br>
$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $
<br><br>
$$ t_{b_i} = \frac{\hat{b_i}}{S_{\hat{b_i}}} $$
<br><br>
$ \alpha = 0.05 $
<br>
$ t_{табл_{0.05}} = 2.110 $
<br>

<br>
$ t_{a} = 1.686 $, $ p\text{-}value_{t_{a}} = 0.110 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.<br>$ t_{b_{1}} = 2.463 $, $ p\text{-}value_{t_{b_{1}}} = 0.025 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.<br>$ t_{b_{2}} = 0.986 $, $ p\text{-}value_{t_{b_{2}}} = 0.338 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.<br>$ t_{b_{3}} = -0.252 $, $ p\text{-}value_{t_{b_{3}}} = 0.804 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.<br>$ t_{b_{4}} = -3.501 $, $ p\text{-}value_{t_{b_{4}}} = 0.003 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.

## Доверительные интервалы

In [51]:
conf_int(model)

$ \hat{b_i} - \hat{\sigma} \cdot t_{табл} \leq b_i \leq \hat{b_i} + \hat{\sigma} \cdot t_{табл} $<br>
<br>
$ \alpha = 0.05 $<br>
<br>
$ a : (-2616.991; 23416.857) $<br>
$ b_{1} : (0.069; 0.898) $<br>
$ b_{2} : (-148.446; 409.076) $<br>
$ b_{3} : (-60.686; 47.750) $<br>
$ b_{4} : (-442.935; -109.810) $

## Дополнительные оценки

In [52]:
other(model)

### Бета-коэффициенты.<br>
<br>
$ \tilde{b_j} = \hat{b_j} \cdot \frac{S_{x_{ij}}}{S_{y_i}} $<br>
<br>
$ \tilde{b_{1}} = 0.000 $<br>
$ \tilde{b_{2}} = 14.363 $<br>
$ \tilde{b_{3}} = -0.139 $<br>
$ \tilde{b_{4}} = -18.200 $<br>
<br>
### Дельта-коэффициенты.<br>
<br>
$ \Delta_j = r_{y_i x_{ij}} \cdot \frac{\hat{b_j}}{R^2} $<br>
<br>
$ \tilde{b_{1}} = 0.421 $<br>
$ \tilde{b_{2}} = 111.584 $<br>
$ \tilde{b_{3}} = -0.545 $<br>
$ \tilde{b_{4}} = 272.419 $<br>
<br>
### Коэффициенты эластичности.<br>
<br>
$ Э_{j} = \hat{b_j} \cdot \frac{\bar{x_{ij}}}{\bar{y_i}} $<br>
<br>
$ \tilde{b_{1}} = 0.159 $<br>
$ \tilde{b_{2}} = 0.281 $<br>
$ \tilde{b_{3}} = -0.051 $<br>
$ \tilde{b_{4}} = -0.134 $

## Интерпретация параметров

Линейная зависимость **объема денежных накоплений в РФ**:
- коэффициент 0.4838 означает, что каждая единица **продажи наличной иностранной
  валюты кредитными организациями физическим лицам** увеличивает **объема денежных
  накоплений в РФ** на 0.4838 каких-то единиц денег.
- коэффициент 130.3151 означает, что рост **курса рубля к доллару США** очень сильно
  увеличивает **объем денежных накоплений в РФ**.
- коэффициент -6.4678 означает, что рост **цены нефти марки "Юралс"** негативно влияет на
  **объем денежных накоплений в РФ**.
- коэффициент -276.3727 означает, что положительная **динамика потребительских цен по
  группам товаров и услуг (месяц к соответствующему месяцу предыдущего года)** значительно
  снижает **объем денежных накоплений в РФ**.
- константа 10400 означает что при полном 'отсутствии экономики' у РФ есть **объема денежных накоплений в РФ** в
  размере 10400 млрд. руб.

# Парная линейная регрессия